In [1]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

import pandas as pd
import pickle

## Import the data
We start by import the data created in the data_collection notebook. This can then be split for training, validation and testing

In [2]:
# Import csv
df = pd.read_csv('data.csv')

# Split data into X and y data
X = df.drop(columns = ['0']).copy().values
y = df['0']

# Split data into testing and training
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8)

## Fit model 
We use a random forest for multiclass classification of the signs.

In [3]:
rf = RandomForestClassifier(class_weight = 'balanced_subsample')
rf.fit(X_train, y_train)
print("Random forest's test score:", rf.score(X_test, y_test))

Random forest's test score: 1.0


## Save the model using pickle

In [4]:
pickle.dump(rf, open('random_forest.sav', 'wb'))